In [74]:
import numpy as np
import sigmf
import scipy
import matplotlib.pyplot as plt
from py3gpp import *

In [75]:
rate = 658 / 1024   # MCS = 16, 64QAM-Table
modulation = '16QAM'
rv = 0
nlayers = 1

A = nrTBS(mod, 1, 25 * 11, 12, R)

cbs_info = nrDLSCHInfo(A, rate)
print(cbs_info)
print(f'tbs = {A} bits, {A // 8} bytes')

in_ = np.zeros((A, 1), np.uint8)
message = "no data available, subframe index = GENERIC"
for i, character in enumerate(message):
    in_[i*8:(i+1)*8, 0] = np.unpackbits(np.uint8(ord(character)), bitorder='big')

tb_in = nrCRCEncode(in_, cbs_info['CRC'])

cbs_in = nrCodeBlockSegmentLDPC(tb_in, cbs_info['BGN'])

enc = nrLDPCEncode(cbs_in, cbs_info['BGN'])

outlen = 13200
ch_in = nrRateMatchLDPC(enc, outlen, rv, modulation, nlayers)
np.savetxt('dummy_pdsch_bits.txt', ch_in, fmt="%1d", delimiter='\n')

{'CRC': '24A', 'L': 24, 'BGN': 1, 'C': 2, 'Lcb': 24, 'F': 312, 'Zc': 208, 'K': 4576, 'N': 13728}
tbs = 8456 bits, 1057 bytes


In [76]:
ch_out = 1-2*(ch_in)

In [77]:
raterec = nrRateRecoverLDPC(ch_out, A, rate, rv, modulation, nlayers)

dec_bits, iters = nrLDPCDecode(raterec, cbs_info['BGN'], 25)
print(f'decoding finished after {iters} iterations')

blk, blk_err = nrCodeBlockDesegmentLDPC(dec_bits, cbs_info['BGN'], A + cbs_info['L'])
print(f'code block CRC: {int(blk_err)}')

out, tb_err = nrCRCDecode(blk, cbs_info['CRC'])
print(f'transport block CRC: {tb_err}')

print(f'output equals input: {np.array_equal(in_, out)}')

rx_bytes = np.packbits(out[:, 0], axis = None, bitorder = 'big')
# print(f'received {rx_bytes.shape[0]} bytes')
id_string = "".join([str(chr(character)) for character in rx_bytes[:50]])
print(f'message = {id_string}')


decoding finished after 25 iterations
code block CRC: 0
transport block CRC: [0]
output equals input: True
message = no data available, subframe index = GENERIC       
